In [153]:
import os
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adagrad
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv3D
%matplotlib inline


In [119]:
import importlib
import DataLoader
importlib.reload(DataLoader)
from DataLoader import DataGenerator

window_size = 20
training_generator = DataGenerator("../data/train", window_size, 32).generate()
test_generator = DataGenerator("../data/test", window_size, 32).generate()

In [123]:
# Sample some statistics over the training set
total_1s = 0
total_0s = 0
for i in range(100):
    sample_batch_in, sample_batch_out = next(training_generator)
    total_1s += np.sum(sample_batch_out == 1)
    total_0s += np.sum(sample_batch_out == 0)
print("Total 1s: %d, total 0s: %d" % (total_1s, total_0s))

Total 1s: 1583, total 0s: 1600


In [154]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=2*(window_size**3)))
model.add(Dense(units=1, activation='sigmoid'))

# For a binary classification problem
model.compile(optimizer=Adagrad(lr=0.01, epsilon=1e-08, decay=0.0),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [155]:
# Train the model, iterating on the data in batches of 32 samples
model.fit_generator(generator=training_generator,
                    steps_per_epoch = 10000,
                    epochs=10,
                    validation_data = test_generator, 
                    validation_steps = 1000)

Epoch 1/10
10000/10000 [==============================] - 61s - loss: 0.0858 - acc: 0.9809 - val_loss: 0.0499 - val_acc: 0.9875
Epoch 2/10
10000/10000 [==============================] - 61s - loss: 0.0356 - acc: 0.9922 - val_loss: 0.0507 - val_acc: 0.9884
Epoch 3/10
10000/10000 [==============================] - 61s - loss: 0.0242 - acc: 0.9954 - val_loss: 0.0363 - val_acc: 0.9921
Epoch 4/10
10000/10000 [==============================] - 60s - loss: 0.0200 - acc: 0.9964 - val_loss: 0.0473 - val_acc: 0.9849
Epoch 5/10
10000/10000 [==============================] - 60s - loss: 0.0202 - acc: 0.9964 - val_loss: 0.0321 - val_acc: 0.9936
Epoch 6/10
10000/10000 [==============================] - 61s - loss: 0.0152 - acc: 0.9973 - val_loss: 0.0291 - val_acc: 0.9943
Epoch 7/10
10000/10000 [==============================] - 61s - loss: 0.0163 - acc: 0.9970 - val_loss: 0.0262 - val_acc: 0.9951
Epoch 8/10
10000/10000 [==============================] - 62s - loss: 0.0122 - acc: 0.9978 - val_loss: 0

In [156]:
import VoxelDistanceFunctionFileLoader
importlib.reload(VoxelDistanceFunctionFileLoader)
from VoxelDistanceFunctionFileLoader import VoxelDistanceFunction

vdf = VoxelDistanceFunction("../data/raw_vdfs/010.vdf")

print(vdf.occupancy.shape, vdf.known.shape, window_size)

center = np.array([5, 5, 50])
x_range = range(center[0], center[0]+window_size)
y_range = range(center[1], center[1]+window_size)
z_range = range(center[2], center[2]+window_size)

example_volume = np.zeros((window_size**3, 2*(window_size**3)))
k = 0
for x in x_range:
    for y in y_range:
        for z in z_range:
            example_volume[k, :] = vdf.GeneratePointwiseSampleAtPoint(window_size, x, y, z)
            k += 1
example_volume_raw = vdf.GeneratePointwiseSampleAtPoint(window_size, center[0], center[1], center[2])

(46, 48, 160) (46, 48, 160) 20


In [157]:
example_predictions = model.predict_on_batch(example_volume)
np.set_printoptions(suppress=True)
print(example_predictions)

[[ 0.00003337]
 [ 0.0000335 ]
 [ 0.00001232]
 ..., 
 [ 0.00025182]
 [ 0.00025182]
 [ 0.00025182]]


In [158]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import ipyvolume
import scipy
def f(x):
    if x == 1:
        v = np.reshape(example_predictions, (window_size, window_size, window_size))
    else:
        v = np.reshape(example_volume_raw[:window_size**3], (window_size, window_size, window_size))
    return ipyvolume.quickvolshow(v, lighting=False, width=800, height=400, data_min=np.nanmin(v), data_max=np.nanmax(v))
interact(f, x=widgets.IntSlider(min=0,max=1,step=1,value=0))

A Jupyter Widget

<function __main__.f>